# MLP Decision Boundary Analysis

| Method                      | What it gives you                    | Limitation                      |
|-----------------------------|--------------------------------------|---------------------------------|
| Grid-based visualization    | Decision boundary in 2D/3D          | Not for high dimensions          |
| Local linear analysis (per activation) | Local boundary, as affine function | Doesn't scale, local only       |
| Polytope enumeration        | Exact regions and boundaries        | Exponential in neurons          |
| Boundary sampling/perturbation | Approximate boundary near examples | Not full boundary, local only   |
| Attribution (e.g., LIME, saliency) | Boundary proximity per example     | Interpretability, not geometry |

Applicability on KANs

## Applicability of Decision Boundary Methods

*   **Grid-based Visualization**
    *   **Applicability:** YES for low dimensional input.
    *   **Reason:** Regardless of activation type, you can sample a grid of input points, evaluate network output, and plot the decision boundary.
    *   **Limitation:** Only practical in 2D/3D and doesn't give full geometric understanding.

*   **Local Linear/Affine Analysis**
    *   **Applicability:** Possible Locally via Taylor Expansion.
    *   **Reason:** For ReLU MLPs, local analysis in each region is exactly linear; for KANs, you can linearize locally via first derivatives (Jacobian).
    *   **Consequence:** The boundary is locally approximated by its tangent hyperplane, but it is not globally polyhedral.

*   **Boundary Sampling / Perturbation**
    *   **Applicability:** Yes.
    *   **How:** Move an input example along a vector until the class changes; the crossing point is on the boundary (regardless of activation type).

*   **Attribution / Saliency Methods**
    *   **Applicability:** Yes.
    *   **Why:** Gradient-based and perturbation-based feature attribution works for any differentiable network (including KANs).

*   **Polytope Enumeration, Linear Region Counting**
    *   **Applicability:** NO, not directly.
    *   **Why:** Polytope enumeration works for piecewise linear nets (like ReLU MLPs), because their boundaries are unions of polyhedra. KANs with general nonlinear activations do not produce polyhedral regions. Their decision boundaries are typically smooth (if activations are smooth).

*   **Distance to Boundary (Adversarial Example)**
    *   **Applicability:** Yes.
    *   **Reason:** This is just optimization—find a small perturbation that changes the class. Applicable to any network.

| Method                      | ReLU MLP | KAN       | Comments                             |
|-----------------------------|----------|-----------|--------------------------------------|
| Grid-based visualization    | Yes      | Yes       | Only in low dimensions                |
| Local linear analysis       | Yes      | Approximate | Linear in ReLU, only approximate in KAN |
| Boundary sampling/perturbation | Yes      | Yes       | Applicable                           |
| Attribution/saliency       | Yes      | Yes       | Applicable                           |
| Polytope enumeration        | Yes      | No        | Intractable for KAN with nonlinear units |
| Distance to boundary        | Yes      | Yes       | Applicable                           |

# Experiments

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

In [2]:
# 2D toy data (make_moons)
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X = X.astype(np.float32)
y = y.astype(np.int32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
class ReLUMLP(nn.Module):
    def __init__(self, n_hidden=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, 1)
        )
    def forward(self, x):
        return self.net(x)

In [4]:
class Spline1D(nn.Module):
    def __init__(self, n_knots=10, xmin=-2.0, xmax=3.0):
        super().__init__()
        self.n_knots = n_knots
        self.xmin = xmin
        self.xmax = xmax
        self.knots = nn.Parameter(torch.linspace(xmin, xmax, n_knots))
        self.values = nn.Parameter(torch.rand(n_knots))  # initial function values

    def forward(self, x):
        # simple linear interpolation; replace with torchinterp or cubic if desired
        x = torch.clamp(x, self.xmin, self.xmax)
        idx = ((x - self.xmin) / (self.xmax - self.xmin) * (self.n_knots - 1)).long()
        idx0 = torch.clamp(idx, 0, self.n_knots - 2)
        idx1 = idx0 + 1
        x0 = self.knots[idx0]
        x1 = self.knots[idx1]
        y0 = self.values[idx0]
        y1 = self.values[idx1]
        t = (x - x0) / (x1 - x0 + 1e-8)
        return y0 + t * (y1 - y0)

class SimpleKAN(nn.Module):
    def __init__(self, n_hidden=16, n_knots=16):
        super().__init__()
        self.lin1 = nn.Linear(2, n_hidden)
        self.acts1 = nn.ModuleList([Spline1D(n_knots) for _ in range(n_hidden)])
        self.lin2 = nn.Linear(n_hidden, n_hidden)
        self.acts2 = nn.ModuleList([Spline1D(n_knots) for _ in range(n_hidden)])
        self.lin3 = nn.Linear(n_hidden, 1)

    def forward(self, x):
        x = self.lin1(x)
        x = torch.stack([act(x[:,i]) for i, act in enumerate(self.acts1)], dim=1)
        x = self.lin2(x)
        x = torch.stack([act(x[:,i]) for i, act in enumerate(self.acts2)], dim=1)
        x = self.lin3(x)
        return x

In [5]:
def train(model, X_train, y_train, epochs=100, lr=1e-2):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    losser = nn.BCEWithLogitsLoss()
    X_tensor = torch.tensor(X_train)
    y_tensor = torch.tensor(y_train).float().unsqueeze(1)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(X_tensor)
        loss = losser(out, y_tensor)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 20 == 0:
            print(f"Epoch {epoch+1}, loss: {loss.item():.4f}")

# Decision Boundary Visualization (Grid-based, 2D)

In [6]:
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    model.eval()
    # grid
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()]).float()
    with torch.no_grad():
        logits = model(grid).cpu().numpy().reshape(xx.shape)
    plt.figure(figsize=(7,5))
    plt.contourf(xx, yy, (logits>0).astype(float), alpha=0.4, levels=[0,0.5,1], cmap='RdBu')
    plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', s=12, edgecolor='k')
    plt.title(title)
    plt.show()

# Boundary Sampling (1D line, "find where class changes")

In [7]:
def boundary_sampling(model, fixed_x=0.0, direction='y', num=200):
    points = []
    coord_vals = np.linspace(-2, 3, num)
    for x1 in coord_vals:
        pt = [fixed_x, x1] if direction=='y' else [x1, fixed_x]
        with torch.no_grad():
            p = torch.tensor(pt).float().unsqueeze(0)
            logit = model(p).squeeze().item()
        label = 1 if logit >= 0 else 0
        points.append((pt, label))
    # Find where label flips
    flips = []
    for i in range(1, len(points)):
        if points[i][1] != points[i-1][1]:
            flips.append(((points[i-1][0], points[i][0])))
    print(f"Boundary crossings along {direction} at fixed {fixed_x}:")
    for a, b in flips:
        print(f"Between {a} and {b}")
    return flips

# Attribution/Saliency (Gradient-based w.r.t. input)

In [8]:
def saliency_map(model, X_sample):
    model.eval()
    X_var = torch.tensor(X_sample).float().unsqueeze(0).requires_grad_(True)
    out = model(X_var)
    out.backward()
    sal = X_var.grad.detach().numpy()[0]
    print("Saliency (input gradient):", sal)
    return sal

# Distance to Boundary (FGSM Adversarial Attack, Norm)

In [9]:
def distance_to_boundary(model, X_sample, y_sample, eps=1e-3):
    # Binary classification, one-step FGSM
    X_var = torch.tensor(X_sample).float().unsqueeze(0).requires_grad_(True)
    y_var = torch.tensor([[y_sample]]).float()
    model.eval()
    out = model(X_var)
    loss = F.binary_cross_entropy_with_logits(out, y_var)
    loss.backward()
    grad = X_var.grad.detach().numpy()[0]
    delta = eps * np.sign(grad)
    perturbed = X_sample + delta
    pred_orig = (torch.sigmoid(out).item() >= 0.5)
    out_pert = model(torch.tensor(perturbed).float().unsqueeze(0))
    pred_new = (torch.sigmoid(out_pert).item() >= 0.5)
    print(f"Origin pred: {pred_orig}, new pred: {pred_new}")
    print(f"Perturbation (l2): {np.linalg.norm(delta):.5f}")
    return perturbed, pred_new

In [10]:
# Train ReLU MLP
relu_mlp = ReLUMLP(n_hidden=32)
train(relu_mlp, X_train, y_train, epochs=200)

# Train KAN
kan = SimpleKAN(n_hidden=32, n_knots=16)
train(kan, X_train, y_train, epochs=200)

# # Plot decision boundaries
# plot_decision_boundary(relu_mlp, X, y, title="ReLU MLP Decision Boundary")
# plot_decision_boundary(kan, X, y, title="KAN Decision Boundary")

# # Boundary sampling example (vertical line x=0)
# boundary_sampling(relu_mlp, fixed_x=0.0, direction='y')
# boundary_sampling(kan, fixed_x=0.0, direction='y')

# # Saliency example
# sample_idx = 0
# sal_map = saliency_map(relu_mlp, X[sample_idx])
# sal_map_kan = saliency_map(kan, X[sample_idx])

# Distance to boundary (adversarial perturbation)
# distance_to_boundary(relu_mlp, X[sample_idx], y[sample_idx])
# distance_to_boundary(kan, X[sample_idx], y[sample_idx])

Epoch 20, loss: 0.2640
Epoch 40, loss: 0.1557
Epoch 60, loss: 0.0887
Epoch 80, loss: 0.0693
Epoch 100, loss: 0.0625
Epoch 120, loss: 0.0603
Epoch 140, loss: 0.0593
Epoch 160, loss: 0.0588
Epoch 180, loss: 0.0585
Epoch 200, loss: 0.0582
Epoch 20, loss: 0.1505
Epoch 40, loss: 3.0365
Epoch 60, loss: 0.2674
Epoch 80, loss: 0.3042
Epoch 100, loss: 0.1599
Epoch 120, loss: 0.1134
Epoch 140, loss: 0.0808
Epoch 160, loss: 0.2772
Epoch 180, loss: 0.0882
Epoch 200, loss: 1.1297


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# For reproducibility
np.random.seed(0)
torch.manual_seed(0)

#------- DATA -------
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X = X.astype(np.float32)
y = y.astype(np.int32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#------- MODELS -------
class ReLUMLP(nn.Module):
    def __init__(self, n_hidden=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, 1)
        )
    def forward(self, x):
        return self.net(x)

class Spline1D(nn.Module):
    def __init__(self, n_knots=10, xmin=-2.0, xmax=3.0):
        super().__init__()
        self.n_knots = n_knots
        self.xmin = xmin
        self.xmax = xmax
        self.knots = nn.Parameter(torch.linspace(xmin, xmax, n_knots), requires_grad=False)
        self.values = nn.Parameter(torch.rand(n_knots))  # initial function values

    def forward(self, x):
        x = torch.clamp(x, self.xmin, self.xmax)
        idx_f = (x - self.xmin) / (self.xmax - self.xmin) * (self.n_knots - 1)
        idx0 = torch.floor(idx_f).long()
        idx1 = torch.clamp(idx0 + 1, max=self.n_knots - 1)
        idx0 = torch.clamp(idx0, max=self.n_knots - 2)
        x0 = self.knots[idx0]
        x1 = self.knots[idx1]
        y0 = self.values[idx0]
        y1 = self.values[idx1]
        t = (x - x0) / (x1 - x0 + 1e-8)
        return y0 + t * (y1 - y0)

class SimpleKAN(nn.Module):
    def __init__(self, n_hidden=32, n_knots=16):
        super().__init__()
        self.lin1 = nn.Linear(2, n_hidden)
        self.acts1 = nn.ModuleList([Spline1D(n_knots) for _ in range(n_hidden)])
        self.lin2 = nn.Linear(n_hidden, n_hidden)
        self.acts2 = nn.ModuleList([Spline1D(n_knots) for _ in range(n_hidden)])
        self.lin3 = nn.Linear(n_hidden, 1)
    def forward(self, x):
        x = self.lin1(x)
        x = torch.stack([act(x[:,i]) for i, act in enumerate(self.acts1)], dim=1)
        x = self.lin2(x)
        x = torch.stack([act(x[:,i]) for i, act in enumerate(self.acts2)], dim=1)
        x = self.lin3(x)
        return x

#------- TRAIN -------
def train(model, X_train, y_train, epochs=150, lr=1e-2, verbose=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    losser = nn.BCEWithLogitsLoss()
    X_tensor = torch.tensor(X_train)
    y_tensor = torch.tensor(y_train).float().unsqueeze(1)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(X_tensor)
        loss = losser(out, y_tensor)
        loss.backward()
        optimizer.step()
        if verbose and (epoch+1) % 50 == 0:
            print(f"Epoch {epoch+1}, loss: {loss.item():.4f}")

# MLP
mlp = ReLUMLP(n_hidden=32)
train(mlp, X_train, y_train, epochs=200)

# KAN
kan = SimpleKAN(n_hidden=32, n_knots=16)
train(kan, X_train, y_train, epochs=200)

#------- UTILITIES -------
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    model.eval()
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 400),
                         np.linspace(y_min, y_max, 400))
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()]).float()
    with torch.no_grad():
        logits = model(grid).cpu().numpy().reshape(xx.shape)
    plt.contourf(xx, yy, (logits>0).astype(float), alpha=0.35, levels=[0,0.5,1], cmap='RdBu')
    plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', s=12, edgecolor='k', alpha=0.8)
    plt.title(title)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

def plot_boundary_sampling(model, X, y, fixed_x=0.0, direction='y', num=200, modelname="Model"):
    points = []
    coord_vals = np.linspace(-2, 3, num)
    for x1 in coord_vals:
        pt = [fixed_x, x1] if direction=='y' else [x1, fixed_x]
        with torch.no_grad():
            p = torch.tensor(pt).float().unsqueeze(0)
            logit = model(p).squeeze().item()
        label = 1 if logit >= 0 else 0
        points.append((pt, label))
    pts = np.array([pt for pt, _ in points])
    labels = np.array([lbl for _, lbl in points])
    plt.figure(figsize=(6,5))
    plot_decision_boundary(model, X, y, title=f"{modelname}: Boundary Sampling (fixed {direction}={fixed_x})")
    if direction=='x':
        plt.plot(coord_vals, np.full_like(coord_vals, fixed_x), 'k--', lw=1)
    else:
        plt.plot(np.full_like(coord_vals, fixed_x), coord_vals, 'k--', lw=1)
    idxs = np.where(np.abs(np.diff(labels))>0)[0]
    for idx in idxs:
        a, b = pts[idx], pts[idx+1]
        plt.plot([a[0], b[0]], [a[1], b[1]], 'go-', markersize=10, label='Boundary Crossing' if idx==idxs[0] else "")
    if len(idxs) > 0:
        plt.legend()
    plt.show()

def plot_saliency_map(model, X, y, X_sample, y_sample, modelname="Model"):
    model.eval()
    X_var = torch.tensor(X_sample).float().unsqueeze(0)
    X_var.requires_grad = True
    out = model(X_var)
    out.backward()
    sal = X_var.grad.detach().numpy()[0]
    plt.figure(figsize=(6,5))
    plot_decision_boundary(model, X, y, title=f"{modelname}: Saliency vector at test pt")
    plt.scatter([X_sample[0]], [X_sample[1]], c=['yellow'], s=80, edgecolor='black', zorder=5, label="Input")
    plt.arrow(X_sample[0], X_sample[1], 0.2*sal[0], 0.2*sal[1], color='k', width=0.007)
    plt.legend()
    plt.show()
    print(f"{modelname} saliency (input gradient) =", np.round(sal,4))

def plot_distance_to_boundary(model, X, y, X_sample, y_sample, modelname="Model", eps=0.2):
    X_var = torch.tensor(X_sample).float().unsqueeze(0).requires_grad_(True)
    y_var = torch.tensor([[y_sample]]).float()
    model.eval()
    out = model(X_var)
    loss = F.binary_cross_entropy_with_logits(out, y_var)
    loss.backward()
    grad = X_var.grad.detach().numpy()[0]
    delta = eps * np.sign(grad)
    perturbed = X_sample + delta
    pred_orig = (torch.sigmoid(out).item() >= 0.5)
    out_pert = model(torch.tensor(perturbed).float().unsqueeze(0))
    pred_new = (torch.sigmoid(out_pert).item() >= 0.5)
    plt.figure(figsize=(6,5))
    plot_decision_boundary(model, X, y, title=f"{modelname}: Adversarial perturbation (eps={eps})")
    plt.scatter([X_sample[0]], [X_sample[1]], c=['yellow'], s=80, label="Original", edgecolor='black', zorder=5)
    plt.scatter([perturbed[0]], [perturbed[1]], c=['red'], s=80, label="Perturbed", edgecolor='black', zorder=5)
    plt.arrow(X_sample[0], X_sample[1], perturbed[0]-X_sample[0], perturbed[1]-X_sample[1], color='black', width=0.01)
    plt.legend()
    plt.show()
    print(f"{modelname}: original pred={int(pred_orig)}, new pred={int(pred_new)}, L2 perturb={np.linalg.norm(delta):.3f}")

#------- PLOTS -------
sample_idx = 0
X_sample = X[sample_idx]
y_sample = y[sample_idx]

#--- MLP
plt.figure(figsize=(7,5))
plot_decision_boundary(mlp, X, y, title="ReLU MLP: Decision Boundary")
plt.show()

plot_boundary_sampling(mlp, X, y, fixed_x=0.0, direction='y', modelname="ReLU MLP")

plot_saliency_map(mlp, X, y, X_sample, y_sample, modelname="ReLU MLP")

plot_distance_to_boundary(mlp, X, y, X_sample, y_sample, modelname="ReLU MLP", eps=0.2)

#--- KAN
plt.figure(figsize=(7,5))
plot_decision_boundary(kan, X, y, title="KAN: Decision Boundary")
plt.show()

plot_boundary_sampling(kan, X, y, fixed_x=0.0, direction='y', modelname="KAN")

plot_saliency_map(kan, X, y, X_sample, y_sample, modelname="KAN")

plot_distance_to_boundary(kan, X, y, X_sample, y_sample, modelname="KAN", eps=0.2)

/tmp/ipykernel_12484/3159951768.py:16: FutureWarning: In the future `np.long` will be defined as the corresponding NumPy scalar.
  y = y.astype(np.long)


AttributeError: module 'numpy' has no attribute 'long'

## Ways to Quantify Expressiveness from the Decision Boundary

1.  **Counting Linear (or Affine) Regions**
    *   **What:** For piecewise (affine) models like ReLU networks, the number of distinct linear regions the decision function splits space into.
    *   **Why:** More regions → more "breaks"/"bends" in the boundary → greater expressiveness.
    *   **How:**
        *   Exact counting is computationally hard for large nets, but tractable in low dimension or for small nets.
        *   Reference: Montúfar et al., 2014; Serra et al., 2018.
        *   There are open-source tools that can do this for small networks.
    *   **Use:** Compare two networks—the one with more regions is (all else equal) more expressive.

2.  **Decision Boundary Length (2D) or Surface Area (Higher d)**
    *   **What:** Compute the length of the boundary curve (in 2D) or surface area (in 3D+) separating classes.
    *   **Why:** More expressive models can create longer (more convoluted) boundaries.
    *   **How (practically in 2D):**
        *   Sample a fine grid.
        *   Extract the boundary as a set of points (e.g., by looking where the predicted class changes).
        *   Estimate total curve length by summing segment distances along the curve.
        *   See also Decision Boundary Complexity from Cavalcanti et al., 2018.
    *   **Use:** Compare models numerically—the longer the boundary, the more intricate.

3.  **Curvature Measures**
    *   **What:** How "bendy" is the boundary? High total (integrated) curvature = more turns and complexity.
    *   **How:**
        *   On a grid, approximate the boundary as a collection of points/segments and estimate local curvature.
        *   Integrate (sum) the curvature along the boundary.
    *   **Use:** Especially relevant for networks with smooth nonlinearities, e.g., KAN, tanh, etc.

4.  **Fractal Dimension of the Boundary (for highly expressive models)**
    *   **What:** For very irregular boundaries, the fractal dimension quantifies self-similarity and complexity at many scales.
    *   **How:** Box-counting or related algorithms.
    *   **References:** See, e.g., Fractal dimension of decision boundaries.
    *   **Use:** High fractal dimension = very high expressiveness.

5.  **Empirical Complexity on Synthetic Data**
    *   **What:** Use challenging synthetic benchmarks (moons, spirals, etc.), train your models, and compare the error rate: who can fit the most complicated boundary?
    *   **Why:** Not a geometric measure, but a proxy for expressiveness.

| Method                      | What is Quantified       | Works for ReLU? | Works for KAN? | Relative Difficulty |
|-----------------------------|--------------------------|-----------------|----------------|---------------------|
| Linear regions              | Piecewise regions        | Yes             | Limited        | Med-hard (exact)    |
| Boundary length/surface     | Complexity of boundary  | Yes             | Yes            | Easy                |
| Curvature                   | Bendiness                | Yes             | Yes            | Med                 |
| Fractal dimension           | Self-similarity          | Yes             | Yes            | Hard                |

In [ ]:
# Assuming you have trained `model` and have data X, y (see previous code)
import numpy as np

def estimate_boundary_length(model, X, resolution=400):
    # Grid in 2D
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_torch = torch.tensor(grid).float()
    with torch.no_grad():
        logits = model(grid_torch).cpu().numpy().reshape(xx.shape)
    preds = (logits > 0).astype(int)
    from skimage import measure
    # Find contours at class boundary
    contours = measure.find_contours(preds, 0.5)
    total_length = 0.0
    for contour in contours:
        # contour is a N x 2 array of [y, x] points
        # Convert to data coordinates
        xs = x_min + (x_max - x_min) * contour[:,1]/(resolution-1)
        ys = y_min + (y_max - y_min) * contour[:,0]/(resolution-1)
        points = np.stack([xs, ys], axis=1)
        seg_lens = np.sqrt(np.sum((points[1:] - points[:-1])**2, axis=1))
        total_length += seg_lens.sum()
    return total_length

# Example:
bl_mlp = estimate_boundary_length(mlp, X)
bl_kan = estimate_boundary_length(kan, X)
print(f"Estimated decision boundary length (MLP): {bl_mlp:.2f}")
print(f"Estimated decision boundary length (KAN): {bl_kan:.2f}")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import numpy as np
import torch
from skimage import measure

def estimate_boundary_length(model, X, resolution=400):
    # Grid in 2D
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_torch = torch.tensor(grid).float()
    with torch.no_grad():
        logits = model(grid_torch).cpu().numpy().reshape(xx.shape)
    preds = (logits > 0).astype(int)
    # Find contours at class boundary
    contours = measure.find_contours(preds, 0.5)
    total_length = 0.0
    for contour in contours:
        # contour is a N x 2 array of [y, x] points
        # Convert to data coordinates
        xs = x_min + (x_max - x_min) * contour[:,1]/(resolution-1)
        ys = y_min + (y_max - y_min) * contour[:,0]/(resolution-1)
        points = np.stack([xs, ys], axis=1)
        seg_lens = np.sqrt(np.sum((points[1:] - points[:-1])**2, axis=1))
        total_length += seg_lens.sum()
    return total_length

# Example:
# Assuming mlp and kan are defined and X is your data
# bl_mlp = estimate_boundary_length(mlp, X)
# bl_kan = estimate_boundary_length(kan, X)
# print(f"Estimated decision boundary length (MLP): {bl_mlp:.2f}")
# print(f"Estimated decision boundary length (KAN): {bl_kan:.2f}")

# To resolve the ModuleNotFoundError:
# You need to install the scikit-image library.
# You can do this using pip:
# pip install scikit-image

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import numpy as np
import torch
from skimage import measure

def estimate_boundary_length(model, X, resolution=400):
    # Grid in 2D
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_torch = torch.tensor(grid).float()
    with torch.no_grad():
        logits = model(grid_torch).cpu().numpy().reshape(xx.shape)
    preds = (logits > 0).astype(int)
    # Find contours at class boundary
    contours = measure.find_contours(preds, 0.5)
    total_length = 0.0
    for contour in contours:
        # contour is a N x 2 array of [y, x] points
        # Convert to data coordinates
        xs = x_min + (x_max - x_min) * contour[:,1]/(resolution-1)
        ys = y_min + (y_max - y_min) * contour[:,0]/(resolution-1)
        points = np.stack([xs, ys], axis=1)
        seg_lens = np.sqrt(np.sum((points[1:] - points[:-1])**2, axis=1))
        total_length += seg_lens.sum()
    return total_length

# Example:
# Assuming mlp and kan are defined and X is your data
# bl_mlp = estimate_boundary_length(mlp, X)
# bl_kan = estimate_boundary_length(kan, X)
# print(f"Estimated decision boundary length (MLP): {bl_mlp:.2f}")
# print(f"Estimated decision boundary length (KAN): {bl_kan:.2f}")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import numpy as np
import torch

def estimate_boundary_length(model, X, resolution=400):
    try:
        from skimage import measure
    except ImportError:
        print("Error: scikit-image is not installed. Please install it using 'pip install scikit-image'")
        return None  # Or raise the exception if you want to stop execution

    # Grid in 2D
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_torch = torch.tensor(grid).float()
    with torch.no_grad():
        logits = model(grid_torch).cpu().numpy().reshape(xx.shape)
    preds = (logits > 0).astype(int)

    # Find contours at class boundary
    contours = measure.find_contours(preds, 0.5)
    total_length = 0.0
    for contour in contours:
        # contour is a N x 2 array of [y, x] points
        # Convert to data coordinates
        xs = x_min + (x_max - x_min) * contour[:,1]/(resolution-1)
        ys = y_min + (y_max - y_min) * contour[:,0]/(resolution-1)
        points = np.stack([xs, ys], axis=1)
        seg_lens = np.sqrt(np.sum((points[1:] - points[:-1])**2, axis=1))
        total_length += seg_lens.sum()
    return total_length

In [ ]:
# Example:
bl_mlp = estimate_boundary_length(mlp, X)
bl_kan = estimate_boundary_length(kan, X)
print(f"Estimated decision boundary length (MLP): {bl_mlp:.2f}")
print(f"Estimated decision boundary length (KAN): {bl_kan:.2f}")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject